# Indices

In [1]:
import pandas as pd
import unicodedata

def remove_accents(input_str: str) -> str:
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])



In [2]:
def slugfy(texto: str) -> str:
    return remove_accents(texto).replace(" ", "-").replace(".", "").lower()

In [4]:
df = pd.read_csv("dados/saude-series.csv") 
df.referencia = df.referencia.apply( lambda x: pd.to_datetime(x, format="%Y%m"))


In [5]:
indices = df.copy()

In [6]:
for x in indices.columns[1:-3]:
    indices[x] = indices[x]/indices.população*1000

In [8]:
indices.to_pickle("dados/indices.pd.pkl")

In [9]:
df.to_pickle("dados/saude-series.pd.pkl")

[{'Asma': 0, 'referencia': Timestamp('2013-08-01 00:00:00')},
 {'Asma': 1, 'referencia': Timestamp('2013-09-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2013-10-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-01-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-02-01 00:00:00')},
 {'Asma': 3, 'referencia': Timestamp('2014-03-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-04-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-05-01 00:00:00')},
 {'Asma': 7, 'referencia': Timestamp('2014-06-01 00:00:00')},
 {'Asma': 28, 'referencia': Timestamp('2014-07-01 00:00:00')},
 {'Asma': 57, 'referencia': Timestamp('2014-08-01 00:00:00')},
 {'Asma': 40, 'referencia': Timestamp('2014-09-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-10-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2014-11-01 00:00:00')},
 {'Asma': 7, 'referencia': Timestamp('2014-12-01 00:00:00')},
 {'Asma': 0, 'referencia': Timestamp('2015-01-01 00:00:00')},
 {'As

# Cidades

In [10]:
import json 

with open("dados/municipios.json") as fp:
    municipios = json.load(fp)

In [21]:
ibge_muni = [(x["pk"], x["fields"]["nome"]) for x in municipios]

with open("dados/lista-ibge-municipio.json","w") as fp:
    json.dump(ibge_muni, fp, ensure_ascii=False)

In [23]:
ibge_regi = [(x["pk"], x["fields"]["regiao"]) for x in municipios]


In [32]:
municipios = pd.DataFrame([(x["pk"], x["fields"]["nome"], x["fields"]["regiao"]) for x in municipios], columns=["IBGE", "municipio", "regiao"])

TypeError: string indices must be integers

In [35]:
municipios.to_pickle("dados/municipios.pd.pkl")

In [36]:
municipios.to_dict(orient="records")

[{'ibge': 4200051, 'municipio': 'Abdon Batista', 'regiao': 'Meio Oeste'},
 {'ibge': 4200101, 'municipio': 'Abelardo Luz', 'regiao': 'Xanxerê'},
 {'ibge': 4200200, 'municipio': 'Agrolândia', 'regiao': 'Alto Vale do Itajaí'},
 {'ibge': 4200309, 'municipio': 'Agronômica', 'regiao': 'Alto Vale do Itajaí'},
 {'ibge': 4200408, 'municipio': 'Água Doce', 'regiao': 'Meio Oeste'},
 {'ibge': 4200507, 'municipio': 'Águas de Chapecó', 'regiao': 'Oeste'},
 {'ibge': 4200556, 'municipio': 'Águas Frias', 'regiao': 'Oeste'},
 {'ibge': 4200606,
  'municipio': 'Águas Mornas',
  'regiao': 'Grande Florianópolis'},
 {'ibge': 4200705,
  'municipio': 'Alfredo Wagner',
  'regiao': 'Grande Florianópolis'},
 {'ibge': 4200754,
  'municipio': 'Alto Bela Vista',
  'regiao': 'Alto Uruguai Catarinense'},
 {'ibge': 4200804, 'municipio': 'Anchieta', 'regiao': 'Extremo Oeste'},
 {'ibge': 4200903, 'municipio': 'Angelina', 'regiao': 'Grande Florianópolis'},
 {'ibge': 4201000,
  'municipio': 'Anita Garibaldi',
  'regiao': '

In [39]:
{ slugfy(x): x for x in municipios.regiao.unique()}

{'meio-oeste': 'Meio Oeste',
 'xanxere': 'Xanxerê',
 'alto-vale-do-itajai': 'Alto Vale do Itajaí',
 'oeste': 'Oeste',
 'grande-florianopolis': 'Grande Florianópolis',
 'alto-uruguai-catarinense': 'Alto Uruguai Catarinense',
 'extremo-oeste': 'Extremo Oeste',
 'serra-catarinense': 'Serra Catarinense',
 'medio-vale-do-itajai': 'Médio Vale do Itajaí',
 'nordeste': 'Nordeste',
 'extremo-sul-catarinense': 'Extremo Sul Catarinense',
 'laguna': 'Laguna',
 'alto-vale-do-rio-do-peixe': 'Alto Vale do Rio do Peixe',
 'foz-do-rio-itajai': 'Foz do Rio Itajaí',
 'planalto-norte': 'Planalto Norte',
 'carbonifera': 'Carbonífera'}

In [ ]:
doen